# Feature Engineering and Iteration

## Setup

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, f1_score
from sklearn.decomposition import PCA
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import time, os, sys, warnings
import wandb
from datetime import datetime
import pickle

from src.config import PROJECT_PATH, DATA_PATH, USE_WANDB, WANDB_PROJECT, WANDB_ENTITY
from src.tracking import ExperimentTracker
from src.training import train_and_evaluate_model, average_f1_score

SEED = 42

In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 10000):
        display(df)

In [3]:
# Initialize the experiment tracker
tracker = ExperimentTracker(
    project_path=PROJECT_PATH,
    use_wandb=USE_WANDB,
    wandb_project_name=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY
)

In [4]:
# Load training data, gesture mappings and inverse gesture mappings
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))

with open('processed_data/gesture_mappings.pkl', 'rb') as f:
    gesture_to_seq_type_map, gesture_map, inv_gesture_map = pickle.load(f)

In [7]:
display_all(df_train.head(2))

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,thm_4,thm_5,tof_1_v0,tof_1_v1,tof_1_v2,tof_1_v3,tof_1_v4,tof_1_v5,tof_1_v6,tof_1_v7,tof_1_v8,tof_1_v9,tof_1_v10,tof_1_v11,tof_1_v12,tof_1_v13,tof_1_v14,tof_1_v15,tof_1_v16,tof_1_v17,tof_1_v18,tof_1_v19,tof_1_v20,tof_1_v21,tof_1_v22,tof_1_v23,tof_1_v24,tof_1_v25,tof_1_v26,tof_1_v27,tof_1_v28,tof_1_v29,tof_1_v30,tof_1_v31,tof_1_v32,tof_1_v33,tof_1_v34,tof_1_v35,tof_1_v36,tof_1_v37,tof_1_v38,tof_1_v39,tof_1_v40,tof_1_v41,tof_1_v42,tof_1_v43,tof_1_v44,tof_1_v45,tof_1_v46,tof_1_v47,tof_1_v48,tof_1_v49,tof_1_v50,tof_1_v51,tof_1_v52,tof_1_v53,tof_1_v54,tof_1_v55,tof_1_v56,tof_1_v57,tof_1_v58,tof_1_v59,tof_1_v60,tof_1_v61,tof_1_v62,tof_1_v63,tof_2_v0,tof_2_v1,tof_2_v2,tof_2_v3,tof_2_v4,tof_2_v5,tof_2_v6,tof_2_v7,tof_2_v8,tof_2_v9,tof_2_v10,tof_2_v11,tof_2_v12,tof_2_v13,tof_2_v14,tof_2_v15,tof_2_v16,tof_2_v17,tof_2_v18,tof_2_v19,tof_2_v20,tof_2_v21,tof_2_v22,tof_2_v23,tof_2_v24,tof_2_v25,tof_2_v26,tof_2_v27,tof_2_v28,tof_2_v29,tof_2_v30,tof_2_v31,tof_2_v32,tof_2_v33,tof_2_v34,tof_2_v35,tof_2_v36,tof_2_v37,tof_2_v38,tof_2_v39,tof_2_v40,tof_2_v41,tof_2_v42,tof_2_v43,tof_2_v44,tof_2_v45,tof_2_v46,tof_2_v47,tof_2_v48,tof_2_v49,tof_2_v50,tof_2_v51,tof_2_v52,tof_2_v53,tof_2_v54,tof_2_v55,tof_2_v56,tof_2_v57,tof_2_v58,tof_2_v59,tof_2_v60,tof_2_v61,tof_2_v62,tof_2_v63,tof_3_v0,tof_3_v1,tof_3_v2,tof_3_v3,tof_3_v4,tof_3_v5,tof_3_v6,tof_3_v7,tof_3_v8,tof_3_v9,tof_3_v10,tof_3_v11,tof_3_v12,tof_3_v13,tof_3_v14,tof_3_v15,tof_3_v16,tof_3_v17,tof_3_v18,tof_3_v19,tof_3_v20,tof_3_v21,tof_3_v22,tof_3_v23,tof_3_v24,tof_3_v25,tof_3_v26,tof_3_v27,tof_3_v28,tof_3_v29,tof_3_v30,tof_3_v31,tof_3_v32,tof_3_v33,tof_3_v34,tof_3_v35,tof_3_v36,tof_3_v37,tof_3_v38,tof_3_v39,tof_3_v40,tof_3_v41,tof_3_v42,tof_3_v43,tof_3_v44,tof_3_v45,tof_3_v46,tof_3_v47,tof_3_v48,tof_3_v49,tof_3_v50,tof_3_v51,tof_3_v52,tof_3_v53,tof_3_v54,tof_3_v55,tof_3_v56,tof_3_v57,tof_3_v58,tof_3_v59,tof_3_v60,tof_3_v61,tof_3_v62,tof_3_v63,tof_4_v0,tof_4_v1,tof_4_v2,tof_4_v3,tof_4_v4,tof_4_v5,tof_4_v6,tof_4_v7,tof_4_v8,tof_4_v9,tof_4_v10,tof_4_v11,tof_4_v12,tof_4_v13,tof_4_v14,tof_4_v15,tof_4_v16,tof_4_v17,tof_4_v18,tof_4_v19,tof_4_v20,tof_4_v21,tof_4_v22,tof_4_v23,tof_4_v24,tof_4_v25,tof_4_v26,tof_4_v27,tof_4_v28,tof_4_v29,tof_4_v30,tof_4_v31,tof_4_v32,tof_4_v33,tof_4_v34,tof_4_v35,tof_4_v36,tof_4_v37,tof_4_v38,tof_4_v39,tof_4_v40,tof_4_v41,tof_4_v42,tof_4_v43,tof_4_v44,tof_4_v45,tof_4_v46,tof_4_v47,tof_4_v48,tof_4_v49,tof_4_v50,tof_4_v51,tof_4_v52,tof_4_v53,tof_4_v54,tof_4_v55,tof_4_v56,tof_4_v57,tof_4_v58,tof_4_v59,tof_4_v60,tof_4_v61,tof_4_v62,tof_4_v63,tof_5_v0,tof_5_v1,tof_5_v2,tof_5_v3,tof_5_v4,tof_5_v5,tof_5_v6,tof_5_v7,tof_5_v8,tof_5_v9,tof_5_v10,tof_5_v11,tof_5_v12,tof_5_v13,tof_5_v14,tof_5_v15,tof_5_v16,tof_5_v17,tof_5_v18,tof_5_v19,tof_5_v20,tof_5_v21,tof_5_v22,tof_5_v23,tof_5_v24,tof_5_v25,tof_5_v26,tof_5_v27,tof_5_v28,tof_5_v29,tof_5_v30,tof_5_v31,tof_5_v32,tof_5_v33,tof_5_v34,tof_5_v35,tof_5_v36,tof_5_v37,tof_5_v38,tof_5_v39,tof_5_v40,tof_5_v41,tof_5_v42,tof_5_v43,tof_5_v44,tof_5_v45,tof_5_v46,tof_5_v47,tof_5_v48,tof_5_v49,tof_5_v50,tof_5_v51,tof_5_v52,tof_5_v53,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,6.214844,3.355469,0.134399,-0.355164,-0.447327,-0.809753,28.943842,31.822186,29.553024,28.592863,28.310535,131.0,134.0,132.0,135.0,98.0,74.0,64.0,60.0,-1.0,-1.0,152.0,153.0,141.0,89.0,68.0,63.0,-1.0,-1.0,-1.0,-1.0,169.0,118.0,86.0,73.0,-1.0,-1.0,-1.0,-1.0,-1.0,147.0,110.0,87.0,126.0,-1.0,-1.0,-1.0,-1.0,-1.0,137.0,108.0,115.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,128.0,110.0,129.0,140.0,-1.0,126.0,131.0,-1.0,-1.0,-1.0,108.0,122.0,139.0,113.0,121.0,-1.0,118.0,96.0,-1.0,-1.0,-1.0,-1.0,165.0,124.0,100.0,102.0,119.0,-1.0,-1.0,

In [ ]:
print(f"Gesture Map: {gesture_map}")
print(f"\nInverted Gesture Map: {inv_gesture_map}")
print(f"\nGesture To Sequence Type: {gesture_to_seq_type_map}")

Gesture Map: {'Cheek - pinch skin': 0, 'Forehead - pull hairline': 1, 'Write name on leg': 2, 'Feel around in tray and pull out an object': 3, 'Neck - scratch': 4, 'Neck - pinch skin': 5, 'Eyelash - pull hair': 6, 'Eyebrow - pull hair': 7, 'Forehead - scratch': 8, 'Above ear - pull hair': 9, 'Wave hello': 10, 'Write name in air': 11, 'Text on phone': 12, 'Pull air toward your face': 13, 'Pinch knee/leg skin': 14, 'Scratch knee/leg skin': 15, 'Drink from bottle/cup': 16, 'Glasses on/off': 17}

Inverted Gesture Map: {0: 'Cheek - pinch skin', 1: 'Forehead - pull hairline', 2: 'Write name on leg', 3: 'Feel around in tray and pull out an object', 4: 'Neck - scratch', 5: 'Neck - pinch skin', 6: 'Eyelash - pull hair', 7: 'Eyebrow - pull hair', 8: 'Forehead - scratch', 9: 'Above ear - pull hair', 10: 'Wave hello', 11: 'Write name in air', 12: 'Text on phone', 13: 'Pull air toward your face', 14: 'Pinch knee/leg skin', 15: 'Scratch knee/leg skin', 16: 'Drink from bottle/cup', 17: 'Glasses on/of

In [7]:
N_SPLITS = 5
SEED = 42

MODEL_PARAMS = {
    'catboost': {
        'objective': 'MultiClass', # CatBoost specific
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'iterations': 1000,
        'learning_rate': 0.05,
        'depth': 6,
        'l2_leaf_reg': 3.0,
        'random_seed': SEED,
        'verbose': False,
        'allow_writing_files': False,
        'task_type' : 'GPU',
    },
    'light_gbm': {
        'objective': 'multiclass',
        'num_class': 18, # Hard coding classes
        'metric': 'multi_logloss',
        'n_estimators': 1000,
        'learning_rate': 0.05,
        'num_leaves': 31,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'bagging_fraction': 0.8,
        'feature_fraction': 0.8,
        'bagging_freq': 1,
        'random_state': SEED,
        'n_jobs': -1,
        'verbose': -1,
    }
}

## Wave 2 Features and Training

In [13]:
# -- Cell: Feature Engineering Function - Wave 2 (REVISED for Continuity) --
def create_wave2_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates Wave 2 features: Adds advanced IMU (magnitude, jerk) and
    Thermopile (gradients) features before performing phase-specific aggregation
    (Wave 1 style).

    Ensures direct continuity with the provided Wave 1 feature engineering logic.
    Correctly uses 'sequence_counter' as an ordering index for calculations like jerk.
    Handles ToF -1.0 values by converting to NaN before aggregation.
    Explicitly manages grouping columns to prevent aggregation errors.
    Aligns with Wave 1's exclusion of thm_0 and tof_0.

    Args:
        df (pd.DataFrame): The raw training dataframe. Must be the SAME as used for Wave 1.

    Returns:
        pd.DataFrame: A dataframe with engineered Wave 2 features, aggregated by phase.
                      Structured identically to Wave 1 output for seamless continuation.
    """
    print("Starting Wave 2 Feature Engineering (Aligned with Wave 1)...")

    # --- State Management: Validate Inputs (Mirroring Wave 1) ---
    # Note: thm_0 and tof_0 are intentionally excluded, matching Wave 1 logic.
    required_cols = ['sequence_id', 'phase', 'subject', 'gesture', 'acc_x', 'acc_y', 'acc_z',
                     'rot_w', 'rot_x', 'rot_y', 'rot_z', 'thm_1', 'thm_2',
                     'thm_3', 'thm_4', 'thm_5', 'sequence_counter']

    # Check for presence of some ToF columns (excluding tof_0)
    tof_cols_exist = any(col.startswith('tof_') and col != 'tof_0' for col in df.columns)
    if not tof_cols_exist:
        print(" Warning: No ToF columns (other than potentially missing 'tof_0') found in DataFrame.")

    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns for Wave 2 (based on Wave 1 structure): {missing_cols}")

    df_feat = df.copy()
    print(f" Input DataFrame shape: {df_feat.shape}")

    # --- 1. Add Advanced IMU Features (Aligned with Wave 1 base structure) ---
    print(" Calculating advanced IMU features...")
    df_feat['acc_mag'] = np.sqrt(df_feat['acc_x']**2 + df_feat['acc_y']**2 + df_feat['acc_z']**2)
    df_feat['rot_mag'] = np.sqrt(df_feat['rot_w']**2 + df_feat['rot_x']**2 + df_feat['rot_y']**2 + df_feat['rot_z']**2)
    print("  - Calculated acc_mag and rot_mag.")

    # Jerk (Derivative of Acceleration Magnitude)
    print(" Calculating acc_mag_jerk using sequence_counter for ordering...")
    df_feat.sort_values(by=['sequence_id', 'sequence_counter'], inplace=True)
    df_feat['acc_mag_jerk'] = df_feat.groupby('sequence_id')['acc_mag'].diff()
    # Note: fillna(0) was used in the original Gemini chat for jerk. We can keep or remove based on strategy.
    # df_feat['acc_mag_jerk'].fillna(0, inplace=True) # Optional, matching original chat
    print("  - Calculated acc_mag_jerk.")

    # --- 2. Add Thermopile Gradient Features (Excluding thm_0, matching Wave 1) ---
    thm_cols_for_grad = ['thm_1', 'thm_2', 'thm_3', 'thm_4'] # Excluding thm_0, thm_5
    print(f" Calculating Thermopile gradients for columns: {thm_cols_for_grad}...")
    for i in range(len(thm_cols_for_grad) - 1):
        col1 = thm_cols_for_grad[i]
        col2 = thm_cols_for_grad[i+1]
        grad_col_name = f"thm_grad_{col1.split('_')[1]}_{col2.split('_')[1]}"
        df_feat[grad_col_name] = df_feat[col2] - df_feat[col1]
    print("  - Calculated Thermopile gradients.")

    # --- 3. Handle -1.0 in ToF columns (if they exist, excluding tof_0) ---
    # Find ToF columns, excluding 'tof_0' as it doesn't exist / is excluded
    tof_columns = [col for col in df_feat.columns if col.startswith('tof_') and col != 'tof_0']
    if tof_columns:
        print(f" Found {len(tof_columns)} ToF columns (excluding 'tof_0'). Handling -1.0 values...")
        df_feat[tof_columns] = df_feat[tof_columns].replace(-1.0, np.nan)
        print(f"  - Replaced -1.0 with NaN in ToF columns.")
    else:
        print(" No ToF columns found to process (excluding 'tof_0').")

    # --- 4. Perform Phase-Specific Aggregation (Directly Mirroring Wave 1) ---
    print(" Performing phase-specific aggregation (Wave 1 style)...")

    # --- Define Aggregations (Including NEW Wave 2 features) ---
    # Base sensor columns (matching Wave 1, excluding thm_0, tof_0)
    imu_cols = ['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']
    thm_cols = ['thm_1', 'thm_2', 'thm_3', 'thm_4'] # Excluding thm_0, thm_5 (matching Wave 1 logic for thm_5)
    # Note: tof_columns list is already defined above, excluding tof_0

    # NEW Wave 2 derived columns to aggregate
    derived_wave2_cols = ['acc_mag', 'rot_mag', 'acc_mag_jerk'] + \
                         [f"thm_grad_{col1.split('_')[1]}_{col2.split('_')[1]}"
                          for i, col1 in enumerate(thm_cols_for_grad[:-1])
                          for col2 in thm_cols_for_grad[i+1:i+2]]

    # Combine all columns for aggregation
    cols_to_aggregate = imu_cols + thm_cols + tof_columns + derived_wave2_cols
    print(f"  Columns identified for aggregation: {len(cols_to_aggregate)}")

    # --- Proactive Debugging: Check for non-numeric columns before aggregation ---
    non_numeric_cols = df_feat[cols_to_aggregate].select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric_cols:
        raise TypeError(f"Non-numeric columns found for aggregation: {non_numeric_cols}. "
                        f"Check data types or exclude these columns. ")

    # Define aggregation functions (matching Wave 1)
    aggregations = ['mean', 'std', 'min', 'max', 'median', 'skew']

    # Create aggregation dictionary
    aggs = {}
    for col in cols_to_aggregate:
        aggs[col] = aggregations

    # Perform the groupby and aggregation (matching Wave 1 structure)
    try:
        print("  - Grouping by ['sequence_id', 'phase'] and aggregating...")
        phase_agg_df = df_feat.groupby(['sequence_id', 'phase']).agg(aggs)
        print(f"  - Aggregation completed. Shape before flattening: {phase_agg_df.shape}")

        # Flatten column names (matching Wave 1)
        print("  - Flattening MultiIndex columns...")
        phase_agg_df.columns = ['_'.join(col).strip() for col in phase_agg_df.columns]
        phase_agg_df.reset_index(inplace=True)
        print(f"  - Columns flattened.")

        # Unstack phase dimension (matching Wave 1)
        print("  - Unstacking 'phase' level...")
        phase_agg_df.set_index(['sequence_id', 'phase'], inplace=True)
        phase_agg_df_unstacked = phase_agg_df.unstack(level='phase')
        print(f"  - Unstacking completed. Shape: {phase_agg_df_unstacked.shape}")

        # Flatten multi-index columns and create corresponding column names (matching Wave 1)
        print("  - Flattening final MultiIndex columns...")
        # Use the exact format from Wave 1: f"{sensor_stat}_{phase}"
        new_cols = [f"{sensor_stat}_{phase}" for sensor_stat, phase in phase_agg_df_unstacked.columns]
        phase_agg_df_unstacked.columns = new_cols
        phase_agg_df_unstacked.reset_index(inplace=True) # sequence_id becomes a column
        print(f"  - Final columns flattened. New column count: {len(phase_agg_df_unstacked.columns)}")

    except KeyError as e:
        raise KeyError(f"Aggregation failed due to missing key/column: {e}. "
                       f"Check if 'phase' or feature columns exist correctly.") from e
    except Exception as e:
        raise RuntimeError(f"An unexpected error occurred during aggregation: {e}") from e

    # --- 5. Merge with Metadata (Directly Mirroring Wave 1) ---
    print(" Merging with metadata (subject, gesture)...")
    # Get unique metadata per sequence_id (matching Wave 1)
    # Note: The original Wave 1 code reset the index of meta_df, which might cause misalignment.
    # A safer approach is to ensure sequence_id is handled correctly.
    # Let's use the same logic but be more explicit about the merge key.
    meta_df = df_feat.groupby('sequence_id')[['subject', 'gesture']].first() # Keep sequence_id as index for now
    # Reset index on the aggregated features to make sequence_id a column for merging
    # phase_agg_df_unstacked should already have sequence_id as a column from reset_index above.
    # Perform the merge explicitly on 'sequence_id'
    final_df = pd.merge(phase_agg_df_unstacked, meta_df, on='sequence_id', how='left')
    print(f"  - Merge completed. Shape: {final_df.shape}")

    # --- Encode Target (Matching Wave 1) ---
    print(" Encoding target variable...")
    # Use the same method as Wave 1 (astype category)
    final_df['gesture_encoded'] = final_df['gesture'].astype('category').cat.codes
    # If a global gesture_map is preferred: final_df['gesture_encoded'] = final_df['gesture'].map(gesture_map)
    print(f"  - Target encoded.")

    print(f"Wave 2 Feature Engineering completed. Shape of features: {final_df.shape}")
    # Ensure column order consistency if needed (e.g., metadata last)
    # meta_cols = ['subject', 'gesture', 'gesture_encoded']
    # other_cols = [c for c in final_df.columns if c not in meta_cols]
    # final_df = final_df[other_cols + meta_cols]

    return final_df


In [9]:
features_df_wave2 = create_wave2_features(df_train)

Starting Wave 2 Feature Engineering (Aligned with Wave 1)...
 Input DataFrame shape: (574945, 341)
 Calculating advanced IMU features...
  - Calculated acc_mag and rot_mag.
 Calculating acc_mag_jerk using sequence_counter for ordering...
  - Calculated acc_mag_jerk.
 Calculating Thermopile gradients for columns: ['thm_1', 'thm_2', 'thm_3', 'thm_4']...
  - Calculated Thermopile gradients.
 Found 320 ToF columns (excluding 'tof_0'). Handling -1.0 values...
  - Replaced -1.0 with NaN in ToF columns.
 Performing phase-specific aggregation (Wave 1 style)...
  Columns identified for aggregation: 337
  - Grouping by ['sequence_id', 'phase'] and aggregating...
  - Aggregation completed. Shape before flattening: (16301, 2022)
  - Flattening MultiIndex columns...
  - Columns flattened.
  - Unstacking 'phase' level...
  - Unstacking completed. Shape: (8151, 4044)
  - Flattening final MultiIndex columns...
  - Final columns flattened. New column count: 4045
 Merging with metadata (subject, gesture

/tmp/ipykernel_110991/3922097889.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phase_agg_df.reset_index(inplace=True)
/tmp/ipykernel_110991/3922097889.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phase_agg_df.reset_index(inplace=True)
/tmp/ipykernel_110991/3922097889.py:131: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

  - Merge completed. Shape: (8151, 4047)
 Encoding target variable...
  - Target encoded.
Wave 2 Feature Engineering completed. Shape of features: (8151, 4048)


In [10]:
display_all(features_df_wave2.head())

sequence_id  acc_x_mean_Gesture  acc_x_mean_Transition  acc_x_std_Gesture  \
0  SEQ_000007            6.875854               5.227969           0.779424   
1  SEQ_000008            3.449929               3.353906           0.736728   
2  SEQ_000013           -7.591276              -6.364640           1.143683   
3  SEQ_000016            5.291590               5.818142           0.940102   
4  SEQ_000018            6.235639               3.881445           0.241211   

   acc_x_std_Transition  acc_x_min_Gesture  acc_x_min_Transition  \
0              1.334370           5.492188              3.613281   
1              1.346681           1.734375              1.886719   
2              1.162073          -9.250000             -7.714844   
3              1.174814           3.859375              3.437500   
4              1.901476           5.718750              1.964844   

   acc_x_max_Gesture  acc_x_max_Transition  acc_x_median_Gesture  \
0           9.015625              7.292969              6.964844   
1           4.949219              5.906250              3.457031   
2          -5.226562             -3.347656             -8.007812   
3           9.378906              7.925781              5.277344   
4           6.832031              6.332031              6.232422   

   acc_x_median_Transition  acc_x_skew_Gesture  acc_x_skew_Transition  \
0                 5.070312            0.671795               0.119323   
1                 2.691406           -0.292417               0.278393   
2                -6.582031            0.395099               1.138648   
3                 6.578125            2.323306              -0.451960   
4                 2.921875            0.194196               0.300721   

   acc_y_mean_Gesture  acc_y_mean_Transition  acc_y_std_Gesture  \
0            5.640137               1.708125           0.582177   
1            7.261364               3.472433           0.824113   
2            3.751562               0.513077           0.571292   
3           -4.744830              -3.984954           0.441674   
4            6.068704               0.779492           0.211266   

   acc_y_std_Transition  acc_y_min_Gesture  acc_y_min_Transition  \
0              3.493182           4.683594             -2.019531   
1              3.638059           4.718750             -0.222656   
2              2.984959           2.656250             -3.273438   
3              0.494274          -5.718750             -4.875000   
4              4.005993           5.566406             -3.164062   

   acc_y_max_Gesture  acc_y_max_Transition  acc_y_median_Gesture  \
0           6.519531              6.214844              5.621094   
1           8.667969              7.976562              7.398438   
2           4.683594              3.957031              3.746094   
3          -3.113281             -2.960938             -4.781250   
4           6.449219              6.718750              6.085938   

   acc_y_median_Transition  acc_y_skew_Gesture  acc_y_skew_Transition  \
0                 0.277344           -0.177613               0.188431   
1                 0.546875           -0.761132               0.295310   
2                 1.695312            0.011108              -0.231933   
3                -3.804688            1.184362               0.267835   
4                -0.791016           -0.492244               0.461042   

   acc_z_mean_Gesture  acc_z_mean_Transition  acc_z_std_Gesture  \
0            4.259521               7.265156           1.247767   
1            5.647609               7.462277           1.740919   
2           -5.493620              -6.818444           0.968642   
3           -6.674517               1.260995           1.942150   
4            4.739890               7.972266           0.455216   

   acc_z_std_Transition  acc_z_min_Gesture  acc_z_min_Transition  \
0              2.335664           1.093750              3.125000   
1              2.752709           1.722656              3.175781   
2  

In [11]:
features_df_wave2.columns[features_df_wave2.columns.str.contains('jerk', case=False)]

Index(['acc_mag_jerk_mean_Gesture', 'acc_mag_jerk_mean_Transition',
       'acc_mag_jerk_std_Gesture', 'acc_mag_jerk_std_Transition',
       'acc_mag_jerk_min_Gesture', 'acc_mag_jerk_min_Transition',
       'acc_mag_jerk_max_Gesture', 'acc_mag_jerk_max_Transition',
       'acc_mag_jerk_median_Gesture', 'acc_mag_jerk_median_Transition',
       'acc_mag_jerk_skew_Gesture', 'acc_mag_jerk_skew_Transition'],
      dtype='object')

In [12]:
FEATURE_WAVE = "Wave2-Revised"

feature_cols_wave2 = [col for col in features_df_wave2.columns if col not in ['sequence_id', 'subject', 'gesture', 'gesture_encoded']]
X_wave2 = features_df_wave2[feature_cols_wave2]
y_wave2 = features_df_wave2['gesture_encoded']
groups_wave2 = features_df_wave2['subject']

In [15]:
# Train models on wave 2 features
results_wave1 = {}
for model_key in ['catboost', 'light_gbm']:
    print(f"\n---- Training {model_key.upper()} on {FEATURE_WAVE} ----")
    EXPERIMENT_NAME = f"{FEATURE_WAVE}-revised-{model_key.upper()}"
    
    result = train_and_evaluate_model(
        X=X_wave2, 
        y=y_wave2, 
        groups=groups_wave2,
        model_type=model_key, 
        model_params=MODEL_PARAMS[model_key],
        cv_splits=N_SPLITS, 
        random_seed=SEED
    )
    results_wave1[model_key] = result
    print(f"{result['model_name']} on {FEATURE_WAVE}: {result['mean_f1']:.4f} (+/- {result['std_f1']:.4f})")
    
    tracker.log_experiment(
    experiment_name=EXPERIMENT_NAME, model_name=model_key.upper(), feature_wave=FEATURE_WAVE,
    cv_score=result['mean_f1'], params=MODEL_PARAMS[model_key],
    notes="Revised loop for wave-2"
    )


---- Training CATBOOST on Wave2-Revised ----
  Starting training and evaluation for CATBOOST...
    Fold 1/5...
      Fold 1 F1 Score: 0.8290
    Fold 2/5...
      Fold 2 F1 Score: 0.7777
    Fold 3/5...
      Fold 3 F1 Score: 0.7707
    Fold 4/5...
      Fold 4 F1 Score: 0.7238
    Fold 5/5...
      Fold 5 F1 Score: 0.7660
  Completed CATBOOST training. Mean F1: 0.7734 (+/- 0.0335)
  Average Fold Time: 181.41s
CATBOOST on Wave2-Revised: 0.7734 (+/- 0.0335)
Experiment 'Wave2-Revised-revised-CATBOOST' logged to ~/code/kaggle/kaggle-cmi-detect-behavior/experiment_log_revised.csv


wandb: Currently logged in as: b-a-chaudhry (b-a-chaudhry-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


cv_score,▁
cv_score,0.77343
feature_wave,Wave2-Revised
model_name,CATBOOST


Experiment 'Wave2-Revised-revised-CATBOOST' logged to W&B

---- Training LIGHT_GBM on Wave2-Revised ----
  Starting training and evaluation for LIGHT_GBM...
    Fold 1/5...
      Fold 1 F1 Score: 0.8207
    Fold 2/5...
      Fold 2 F1 Score: 0.7776
    Fold 3/5...
      Fold 3 F1 Score: 0.7698
    Fold 4/5...
      Fold 4 F1 Score: 0.7232
    Fold 5/5...
      Fold 5 F1 Score: 0.7638
  Completed LIGHT_GBM training. Mean F1: 0.7710 (+/- 0.0311)
  Average Fold Time: 98.84s
LIGHT_GBM on Wave2-Revised: 0.7710 (+/- 0.0311)
Experiment 'Wave2-Revised-revised-LIGHT_GBM' logged to ~/code/kaggle/kaggle-cmi-detect-behavior/experiment_log_revised.csv


cv_score,▁
cv_score,0.77103
feature_wave,Wave2-Revised
model_name,LIGHT_GBM


Experiment 'Wave2-Revised-revised-LIGHT_GBM' logged to W&B


## Wave 3 Features and Training

In [16]:
def create_wave3_pca_features(df: pd.DataFrame, n_components: int = 50, random_state: int = SEED) -> pd.DataFrame:
    """
    Creates Wave 3 features: Applies PCA to the ToF sensor data from the raw dataframe,
    then merges the resulting PCA components with the features from Wave 2.

    This function correctly handles the full pipeline:
    1.  Generates Wave 2 features (which includes Wave 1 features implicitly).
    2.  Extracts and preprocesses ToF data from the original raw dataframe.
    3.  Applies PCA to the ToF data, grouped by sequence_id and phase.
    4.  Merges the PCA components back onto the Wave 2 features.

    Correctly uses 'sequence_counter' implicitly via Wave 2 for any ordering needs.
    Handles ToF -1.0 values by converting to NaN before PCA.
    Ensures continuity with Waves 1 and 2.
    Explicitly manages grouping columns to avoid aggregation errors during PCA prep.
    Notes that tof_0 does not exist in the training dataset.

    Args:
        df (pd.DataFrame): The raw training dataframe containing sensor data,
                           sequence_id, phase, subject, gesture, and sequence_counter.
                           This MUST be the same raw df used for Waves 1 & 2.
        n_components (int, optional): Number of PCA components to generate per phase.
                                      Defaults to 50.
        random_state (int, optional): Random state for PCA. Defaults to 42.

    Returns:
        pd.DataFrame: A dataframe with Wave 2 features plus new PCA features,
                      aggregated to the sequence_id level. Ready for model training.
    """
    print("Starting Wave 3 Feature Engineering (ToF PCA)...")
    
    # --- Proactive State Management: Validate Input ---
    required_base_cols = [
        'sequence_id', 'phase', 'subject', 'gesture', 'sequence_counter'
    ]
    # Check for presence of other ToF columns (assuming they start with 'tof_' and skip tof_0)
    tof_cols_exist = any(col.startswith('tof_') and col != 'tof_0' for col in df.columns)

    if not tof_cols_exist:
         raise ValueError("No ToF columns (other than potentially missing 'tof_0') found in input DataFrame. "
                          "PCA on ToF cannot proceed.")
         
    missing_cols = [col for col in required_base_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required core columns for Wave 3: {missing_cols}")
    
    df_raw = df.copy() # Keep original for ToF PCA
    # --- 1. Generate Wave 2 Features ---
    print(" Generating Wave 2 features (includes Wave 1)...")
    # This step ensures continuity and provides the base feature set.
    # It also correctly handles -1.0 in ToF (for the aggregated features) and sequence_counter.
    wave2_features_df = create_wave2_features(df_raw)
    print(f" Wave 2 features generated. Shape: {wave2_features_df.shape}")
    
    # --- 2. Prep ToF for PCA ---
    tof_columns = [col for col in df_raw.columns if col.startswith('tof_')]
    if not tof_columns:
        raise ValueError("No ToF columns found for PCA after excluding 'tof_0'.")
    
    print(f"  - Found {len(tof_columns)} ToF columns for PCA: {tof_columns[:5]}...")
    
    # Handle -1.0 in ToF columns (in the raw data for PCA)
    print("  - Handling -1.0 values in ToF data for PCA...")
    df_tof_for_pca = df_raw[['sequence_id', 'phase'] + tof_columns].copy()
    initial_nans_before = df_tof_for_pca[tof_columns].isna().sum().sum()
    df_tof_for_pca[tof_columns] = df_tof_for_pca[tof_columns].replace(-1.0, np.nan)
    nans_introduced = df_tof_for_pca[tof_columns].isna().sum().sum() - initial_nans_before
    print(f"  - Replaced {nans_introduced} instances of -1.0 with NaN in ToF data for PCA.")
    
    # --- 3. Apply PCA per Phase ---
    print(f" Applying PCA to ToF data for each phase (n_components={n_components})...")
    pca_results = []
    phases = df_tof_for_pca['phase'].unique()
    print(f"  - Phases found: {phases}")
    
    pca_models = {}
    
    for phase in phases:
        print(f"     - Processing phase: {phase}")
        df_phase = df_tof_for_pca[df_tof_for_pca['phase'] == phase].copy()
        sequence_ids = df_phase['sequence_id']
        
        # Drop non-feature columns for PCA fitting
        X_phase = df_phase.drop(columns=['sequence_id', 'phase'])
        
        # Proactive debugging : Check for non-numeric data
        non_numeric_tof_cols = X_phase.select_dtypes(exclude=[np.number]).columns.tolist()
        if non_numeric_tof_cols:
            raise TypeError(f"Non-numeric ToF columns found for PCA in phase {phase}: {non_numeric_tof_cols}. "
                             f"Check data types. ")
            
        # --- Impute NaN values before PCA ---
        # PCA cannot handle NaN. We need to impute them.
        # A simple and common strategy is to fill with 0.
        #### Alternative strategies (mean, median imputation per column) could be explored later.
        nan_count_before_impute = X_phase.isna().sum().sum()
        if nan_count_before_impute > 0:
            print(f"    - Found {nan_count_before_impute} NaN values in phase '{phase}'. Imputing with 0...")
            X_phase_imputed = X_phase.fillna(0) # <--- IMPTATION STEP ADDED ---
        else:
            print(f"    - No NaN values found in phase '{phase}'.")
            X_phase_imputed = X_phase
        
        # Fit PCA
        pca = PCA(n_components=n_components, random_state=random_state)
        try:
            X_pca_phase = pca.fit_transform(X_phase_imputed)
        except ValueError as e: # Catch any other potential ValueError from PCA
            raise RuntimeError(f"PCA failed unexpectedly for phase '{phase}': {e}") from e
            
        # Store PCA model
        pca_models[phase] = pca

        # Create DataFrame for PCA results of this phase
        pca_columns = [f"tof_pca_{i+1}_{phase}" for i in range(n_components)]
        df_pca_phase = pd.DataFrame(X_pca_phase, columns=pca_columns, index=sequence_ids).reset_index() # Add sequence_id back
        
        pca_results.append(df_pca_phase)
        print(f"   - PCA completed for phase {phase}. Components: {X_pca_phase.shape[1]}")

    if not pca_results:
        raise RuntimeError("PCA process failed to generate results for any phase.")

    # --- 4. Combine PCA Features from all Phases ---
    print(" Combining PCA features from all phases...")
    # Concatenate PCA results from different phases
    # We need to merge them on 'sequence_id' because each df_pca_phase has the same sequence_ids
    # but different columns (based on phase).
    combined_pca_df = pca_results[0]
    for df_pca_phase in pca_results[1:]:
        # Use a left join to ensure all sequence_ids from the first phase are kept
        # If a sequence_id doesn't have data for a later phase, it will get NaNs for those PCA features.
        # This is consistent with how phase-specific aggregates work.
        combined_pca_df = pd.merge(combined_pca_df, df_pca_phase, on='sequence_id', how='outer')
        
    combined_pca_df.set_index('sequence_id', inplace=True)
    print(f"  - Combined PCA features shape: {combined_pca_df.shape}")
    print(f"  - Combined PCA columns: {list(combined_pca_df.columns)[:10]}...")

    # --- 5. Merge PCA Features with Wave 2 Features ---
    print(" Merging PCA features with Wave 2 features...")
    # wave2_features_df index is 'sequence_id'
    # combined_pca_df index is 'sequence_id'
    try:
        # Use 'outer' join to be safe, though 'left' (Wave2 as base) is likely equivalent
        # given how the data is structured. 'outer' ensures no sequence_ids are lost.
        final_features_df = pd.merge(wave2_features_df, combined_pca_df, left_index=True, right_index=True, how='left')
        # Note: If a sequence_id in wave2_features_df had no data for a specific phase,
        # the corresponding PCA features for that phase will be NaN. This is expected.
    except Exception as e:
         raise RuntimeError(f"Failed to merge Wave 2 features with PCA features: {e}") from e

    print(f" Final Wave 3 DataFrame shape: {final_features_df.shape}")
    print("Wave 3 Feature Engineering (ToF PCA) completed successfully.")
    # Optional: Store pca_models in an attribute or return them if needed for inference
    # final_features_df._pca_models = pca_models 
    return final_features_df
    

In [17]:
features_df_wave3 = create_wave3_pca_features(df_train, n_components=50)
print(f"Wave 3 features shape: {features_df_wave3.shape}")

Starting Wave 3 Feature Engineering (ToF PCA)...
 Generating Wave 2 features (includes Wave 1)...
Starting Wave 2 Feature Engineering (Aligned with Wave 1)...
 Input DataFrame shape: (574945, 341)
 Calculating advanced IMU features...
  - Calculated acc_mag and rot_mag.
 Calculating acc_mag_jerk using sequence_counter for ordering...
  - Calculated acc_mag_jerk.
 Calculating Thermopile gradients for columns: ['thm_1', 'thm_2', 'thm_3', 'thm_4']...
  - Calculated Thermopile gradients.
 Found 320 ToF columns (excluding 'tof_0'). Handling -1.0 values...
  - Replaced -1.0 with NaN in ToF columns.
 Performing phase-specific aggregation (Wave 1 style)...
  Columns identified for aggregation: 337
  - Grouping by ['sequence_id', 'phase'] and aggregating...
  - Aggregation completed. Shape before flattening: (16301, 2022)
  - Flattening MultiIndex columns...
  - Columns flattened.
  - Unstacking 'phase' level...
  - Unstacking completed. Shape: (8151, 4044)
  - Flattening final MultiIndex colum

/tmp/ipykernel_38254/3922097889.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phase_agg_df.reset_index(inplace=True)
/tmp/ipykernel_38254/3922097889.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phase_agg_df.reset_index(inplace=True)
/tmp/ipykernel_38254/3922097889.py:131: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fram

  - Merge completed. Shape: (8151, 4047)
 Encoding target variable...
  - Target encoded.
Wave 2 Feature Engineering completed. Shape of features: (8151, 4048)
 Wave 2 features generated. Shape: (8151, 4048)
  - Found 320 ToF columns for PCA: ['tof_1_v0', 'tof_1_v1', 'tof_1_v2', 'tof_1_v3', 'tof_1_v4']...
  - Handling -1.0 values in ToF data for PCA...
  - Replaced 105770214 instances of -1.0 with NaN in ToF data for PCA.
 Applying PCA to ToF data for each phase (n_components=50)...
  - Phases found: ['Transition' 'Gesture']
     - Processing phase: Transition
    - Found 66892478 NaN values in phase 'Transition'. Imputing with 0...
   - PCA completed for phase Transition. Components: 50
     - Processing phase: Gesture
    - Found 42400168 NaN values in phase 'Gesture'. Imputing with 0...
   - PCA completed for phase Gesture. Components: 50
 Combining PCA features from all phases...
  - Combined PCA features shape: (10170422, 100)
  - Combined PCA columns: ['tof_pca_1_Transition', 'to